In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from scipy.stats import chi2_contingency
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.formula.api import ols

In [2]:
df=pd.read_csv('watson_healthcare_modified.csv')

In [3]:
df.shape

(1676, 35)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1676 entries, 0 to 1675
Data columns (total 35 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   EmployeeID                1676 non-null   int64 
 1   Age                       1676 non-null   int64 
 2   Attrition                 1676 non-null   object
 3   BusinessTravel            1676 non-null   object
 4   DailyRate                 1676 non-null   int64 
 5   Department                1676 non-null   object
 6   DistanceFromHome          1676 non-null   int64 
 7   Education                 1676 non-null   int64 
 8   EducationField            1676 non-null   object
 9   EmployeeCount             1676 non-null   int64 
 10  EnvironmentSatisfaction   1676 non-null   int64 
 11  Gender                    1676 non-null   object
 12  HourlyRate                1676 non-null   int64 
 13  JobInvolvement            1676 non-null   int64 
 14  JobLevel                

In [5]:
df.describe()

,EmployeeID,Age,DailyRate,DistanceFromHome,Education,EmployeeCount,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,...,RelationshipSatisfaction,StandardHours,Shift,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
count,1.676000e+03,1676.000000,1676.000000,1676.000000,1676.000000,1676.0,1676.000000,1676.000000,1676.000000,1676.000000,...,1676.000000,1676.0,1676.000000,1676.000000,1676.000000,1676.000000,1676.000000,1676.000000,1676.000000,1676.000000
mean,1.456796e+06,36.866348,800.557876,9.221957,2.907518,1.0,2.714797,65.470167,2.724940,2.066826,...,2.718377,80.0,0.806086,11.338902,2.805489,2.766110,7.033413,4.264916,2.200477,4.135442
std,2.487486e+05,9.129126,401.594438,8.158118,1.025835,0.0,1.097534,20.207572,0.714121,1.113423,...,1.078162,0.0,0.855527,7.834996,1.288431,0.702369,6.098991,3.627456,3.229587,3.559662
min,1.025177e+06,18.000000,102.000000,1.000000,1.000000,1.0,1.000000,30.000000,1.000000,1.000000,...,1.000000,80.0,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,1.235832e+06,30.000000,465.000000,2.000000,2.000000,1.0,2.000000,48.000000,2.000000,1.000000,...,2.000000,80.0,0.000000,6.000000,2.000000,2.000000,3.000000,2.000000,0.000000,2.000000
50%,1.464606e+06,36.000000,796.500000,7.000000,3.000000,1.0,3.000000,65.500000,3.000000,2.000000,...,3.000000,80.0,1.000000,10.000000,3.000000,3.000000,5.000000,3.000000,1.000000,3.000000
75%,1.667992e+06,43.000000,1157.000000,14.000000,4.000000,1.0,4.000000,83.000000,3.000000,3.000000,...,4.000000,80.0,1.000000,15.000000,3.000000,3.000000,10.000000,7.000000,3.000000,7.000000
max,1.886378e+06,60.000000,1499.000000,29.000000,5.000000,1.0,4.000000,100.000000,4.000000,5.000000,...,4.000000,80.0,3.000000,40.000000,6.000000,4.000000,40.000000,18.000000,15.000000,17.000000


In [6]:
pd.set_option('display.max_columns',None)
df.head()

,EmployeeID,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NumCompaniesWorked,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,Shift,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,1313919,41,No,Travel_Rarely,1102,Cardiology,1,2,Life Sciences,1,2,Female,94,3,2,Nurse,4,Single,5993,19479,8,Y,Yes,11,3,1,80,0,8,0,1,6,4,0,5
1,1200302,49,No,Travel_Frequently,279,Maternity,8,1,Life Sciences,1,3,Male,61,2,2,Other,2,Married,5130,24907,1,Y,No,23,4,4,80,1,10,3,3,10,7,1,7
2,1060315,37,Yes,Travel_Rarely,1373,Maternity,2,2,Other,1,4,Male,92,2,1,Nurse,3,Single,2090,2396,6,Y,Yes,15,3,2,80,0,7,3,3,0,0,0,0
3,1272912,33,No,Travel_Frequently,1392,Maternity,3,4,Life Sciences,1,4,Female,56,3,1,Other,3,Married,2909,23159,1,Y,Yes,11,3,3,80,0,8,3,3,8,7,3,0
4,1414939,27,No,Travel_Rarely,591,Maternity,2,1,Medical,1,1,Male,40,3,1,Nurse,2,Married,3468,16632,9,Y,No,12,3,4,80,1,6,3,3,2,2,2,2


In [7]:
df.drop('EmployeeID',inplace=True,axis=1)

In [8]:
df.isnull().sum()

Age                         0
Attrition                   0
BusinessTravel              0
DailyRate                   0
Department                  0
DistanceFromHome            0
Education                   0
EducationField              0
EmployeeCount               0
EnvironmentSatisfaction     0
Gender                      0
HourlyRate                  0
JobInvolvement              0
JobLevel                    0
JobRole                     0
JobSatisfaction             0
MaritalStatus               0
MonthlyIncome               0
MonthlyRate                 0
NumCompaniesWorked          0
Over18                      0
OverTime                    0
PercentSalaryHike           0
PerformanceRating           0
RelationshipSatisfaction    0
StandardHours               0
Shift                       0
TotalWorkingYears           0
TrainingTimesLastYear       0
WorkLifeBalance             0
YearsAtCompany              0
YearsInCurrentRole          0
YearsSinceLastPromotion     0
YearsWithC

## Smote

In [9]:
df.Attrition.value_counts()

No     1477
Yes     199
Name: Attrition, dtype: int64

In [10]:
no=df[df.Attrition=='No']
yes=df[df.Attrition=='Yes']

from sklearn.utils import resample

over=resample(yes,replace=True,n_samples=len(no),random_state=23)

dfsmote=pd.concat([no,over])
dfsmote.Attrition.value_counts()

No     1477
Yes    1477
Name: Attrition, dtype: int64

In [11]:
df.shape

(1676, 34)

# 5. Data Preprocessing

In [12]:

x = dfsmote.drop("Attrition",axis = 1)
y =  dfsmote["Attrition"]

In [13]:
x.shape

(2954, 33)

In [14]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()

In [15]:
df.Attrition=le.fit_transform(df.Attrition)

In [16]:
p=pd.get_dummies(x.BusinessTravel,drop_first=True)
o=pd.get_dummies(x.Department,drop_first=True)
i=pd.get_dummies(x.JobRole,drop_first=True)
u=pd.get_dummies(x.OverTime,drop_first=True)
y=pd.get_dummies(x.MaritalStatus,drop_first=True)
f=pd.get_dummies(x.JobRole,drop_first=True)
h=pd.get_dummies(x.OverTime,drop_first=True)
j=pd.get_dummies(x.MaritalStatus,drop_first=True)

In [17]:
x=pd.concat([p,o,i,u,f,j,h,df],axis=1)

In [18]:
x.drop(['BusinessTravel','Department','JobRole','OverTime','MaritalStatus','Over18','Gender','EducationField'],axis=1,inplace=True)

In [19]:
x.head()

,Travel_Frequently,Travel_Rarely,Maternity,Neurology,Administrative,Nurse,Other,Therapist,Yes,Administrative,Nurse,Other,Therapist,Married,Single,Yes,Age,Attrition,DailyRate,DistanceFromHome,Education,EmployeeCount,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,JobSatisfaction,MonthlyIncome,MonthlyRate,NumCompaniesWorked,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,Shift,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,1,1,41,0,1102,1,2,1,2,94,3,2,4,5993,19479,8,11,3,1,80,0,8,0,1,6,4,0,5
1,1,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,49,0,279,8,1,1,3,61,2,2,2,5130,24907,1,23,4,4,80,1,10,3,3,10,7,1,7
3,1,0,1,0,0,0,1,0,1,0,0,1,0,1,0,1,33,0,1392,3,4,1,4,56,3,1,3,2909,23159,1,11,3,3,80,0,8,3,3,8,7,3,0
4,0,1,1,0,0,1,0,0,0,0,1,0,0,1,0,0,27,0,591,2,1,1,1,40,3,1,2,3468,16632,9,12,3,4,80,1,6,3,3,2,2,2,2
5,1,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,32,0,1005,2,2,1,4,79,3,1,4,3068,11864,0,13,3,3,80,0,8,2,2,7,7,3,6


In [20]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
x[['MonthlyRate','MonthlyIncome','HourlyRate','Age','DailyRate']] = scaler.fit_transform(x[['MonthlyRate','Age','MonthlyIncome','HourlyRate','DailyRate']])

In [21]:
x.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2954 entries, 0 to 1165
Data columns (total 42 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Travel_Frequently         2954 non-null   uint8  
 1   Travel_Rarely             2954 non-null   uint8  
 2   Maternity                 2954 non-null   uint8  
 3   Neurology                 2954 non-null   uint8  
 4   Administrative            2954 non-null   uint8  
 5   Nurse                     2954 non-null   uint8  
 6   Other                     2954 non-null   uint8  
 7   Therapist                 2954 non-null   uint8  
 8   Yes                       2954 non-null   uint8  
 9   Administrative            2954 non-null   uint8  
 10  Nurse                     2954 non-null   uint8  
 11  Other                     2954 non-null   uint8  
 12  Therapist                 2954 non-null   uint8  
 13  Married                   2954 non-null   uint8  
 14  Single  

In [22]:
x

,Travel_Frequently,Travel_Rarely,Maternity,Neurology,Administrative,Nurse,Other,Therapist,Yes,Administrative,Nurse,Other,Therapist,Married,Single,Yes,Age,Attrition,DailyRate,DistanceFromHome,Education,EmployeeCount,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,JobSatisfaction,MonthlyIncome,MonthlyRate,NumCompaniesWorked,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,Shift,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,1,1,0.914286,0,0.715820,1,2,1,2,0.262454,3,2,4,0.547619,0.698053,8,11,3,1,80,0,8,0,1,6,4,0,5
1,1,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0.442857,0,0.126700,8,1,1,3,0.217009,2,2,2,0.738095,0.916001,1,23,4,4,80,1,10,3,3,10,7,1,7
3,1,0,1,0,0,0,1,0,1,0,0,1,0,1,0,1,0.371429,0,0.923407,3,4,1,4,0.100053,3,1,3,0.357143,0.845814,1,11,3,3,80,0,8,3,3,8,7,3,0
4,0,1,1,0,0,1,0,0,0,0,1,0,0,1,0,0,0.142857,0,0.350036,2,1,1,1,0.129489,3,1,2,0.214286,0.583738,9,12,3,4,80,1,6,3,3,2,2,2,2
5,1,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0.700000,0,0.646385,2,2,1,4,0.108425,3,1,4,0.333333,0.392291,0,13,3,3,80,0,8,2,2,7,7,3,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1278,0,1,0,0,0,1,0,0,1,0,1,0,0,0,1,1,0.771429,1,0.143880,27,3,1,3,0.252975,3,2,4,0.404762,0.457659,1,18,3,4,80,0,10,2,3,10,7,7,7
456,1,0,0,0,0,0,1,0,1,0,0,1,0,0,1,1,0.557143,1,0.861847,5,3,1,2,0.045761,3,1,2,0.000000,0.239510,1,14,3,4,80,0,0,3,3,0,0,0,0
1080,0,1,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0.800000,1,0.410880,9,4,1,4,0.053133,3,1,1,0.523810,0.792491,3,14,3,2,80,0,15,3,1,5,4,1,0
51,0,1,1,0,0,1,0,0,1,0,1,0,0,0,1,1,0.285714,1,0.953472,5,4,1,3,0.128067,3,1,3,0.238095,0.364786,1,13,3,3,80,0,2,3,2,2,2,2,2


In [23]:
from sklearn.decomposition import PCA
pca=PCA()
x_pca=pca.fit_transform(x)

In [24]:

from sklearn.decomposition import PCA
pca = PCA().fit(x)

fig, ax = plt.subplots()
xi = np.arange(1, 14, step=1)
y = np.cumsum(pca.explained_variance_ratio_)

plt.ylim(0.0,1.1)

plt.xlabel('Number of Components')
plt.xticks(np.arange(0, , step=1)) #change from 0-based array index to 1-based human-readable label
plt.ylabel('Cumulative variance (%)')
plt.title('The number of components needed to explain variance')

plt.axhline(y=0.95, color='r', linestyle='-')
plt.text(0.5, 0.85, '95% cut-off threshold', color = 'red', fontsize=16)

ax.grid(axis='x')
plt.show()

SyntaxError: invalid syntax (2109887029.py, line 11)

In [ ]:
x_pca.shape

In [ ]:
ps = pd.DataFrame(x_pca)
ps.head()

In [ ]:
from sklearn.model_selection import train_test_split,GridSearchCV

from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn.metrics import classification_report, confusion_matrix,accuracy_score
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn import metrics

In [ ]:
xtrain,xtest,ytrain,ytest=train_test_split(x_pca,y,test_size=0.3)

In [ ]:
rf=RandomForestClassifier(n_estimators=1000)
rf.fit(xtrain,ytrain)

In [ ]:
y_pred=rf.predict(xtest)

In [ ]:
accuracy_score(ytest,y_pred)

In [ ]:
rf.score(xtest,ytest)

In [ ]:
print(classification_report(ytest,y_pred))

In [ ]:
from sklearn.tree import DecisionTreeClassifier


In [ ]:
dt=DecisionTreeClassifier(max_depth=1000)
dt.fit(xtrain,ytrain)

In [ ]:
y_pred=dt.predict(xtest)

In [ ]:
accuracy_score(ytest,y_pred)

In [ ]:
#classification Report
print(classification_report(ytest,y_pred))